In [2]:
!nvitop

)07=Sun Aug 04 14:46:59 2024
╒═════════════════════════════════════════════════════════════════════════════╕
│ NVITOP 1.3.2        Driver Version: 550.100       CUDA Driver Version: 12.4 │
├───────────────────────────────┬──────────────────────┬──────────────────────┤
│ GPU  Name        Persistence-M│ Bus-Id        Disp.A │ Volatile Uncorr. ECC │
│ Fan  Temp  Perf  Pwr:Usage/Cap│         Memory-Usage │ GPU-Util  Compute M. │
╞═══════════════════════════════╪══════════════════════╪══════════════════════╡
│   0  GeForce RTX 3090    On   │ 00000000:C3:00.0 Off │                  N/A │
│ 86%   65C    P8    36W / 350W │   9642MiB / 24.00GiB │      0%      Default │
├───────────────────────────────┼──────────────────────┼──────────────────────┤
│   1  GeForce RTX 3090    On   │ 00000000:E1:00.0 Off │                  N/A │
│ 44%   59C    P8    18W / 350W │   9642MiB / 24.00GiB │      0%      Default │
╘═══════════════════════════════╧══════════════════════╧══════════════════════╛
[ CPU: ▎

In [4]:
from transformer_lens import HookedTransformer

model = HookedTransformer.from_pretrained("NeelNanda/GELU_1L512W_C4_Code", device = 'cuda')

/workspace/venv/lib/python3.10/site-packages/transformer_lens/utils.py:62: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(file_path, map_location="cpu")


Loaded pretrained model NeelNanda/GELU_1L512W_C4_Code into HookedTransformer


In [6]:
model.cfg.d_mlp

2048

In [2]:
from circuit import IOICircuit
import json

with open('tasks/ioi/cfg.json') as f:
    cfg = json.load(f)

with open('tasks/ioi/names.json') as f:
    names = json.load(f)

with open('tasks/ioi/prompts.json') as f:
    prompts = json.load(f)

circuit = IOICircuit(
    model= model,
    cfg=cfg, 
    names=names, 
    prompts=prompts)

In [3]:
p = circuit.prompts[0]
p.text, p.variables

('Then, Bella and John had lots of fun at the clinic. John gave a pillow to',
 {'IO': ' Bella',
  'POS': 'ABB',
  'END': 18,
  'IO_pos': 3,
  'S1_pos': 5,
  'S2_pos': 14,
  'S': ' John'})

In [4]:
p.generate_corrupted('POS', model).text

'Then, John and Bella had lots of fun at the clinic. John gave a pillow to'

In [5]:
circuit.create_task_df()
circuit.task_df.head()

,prompt,IO,S,POS,IO_pos,S1_pos,S1+1_pos,S2_pos,END
0,"Then, Bella and John had lots of fun at the cl...",Bella,John,ABB,3,5,6,14,18
1,"Then, Ross and Adrian were working at the jers...",Ross,Adrian,ABB,3,5,6,12,18
2,"Then, Paul and Carl had lots of fun at the man...",Paul,Carl,ABB,3,5,6,14,18
3,"Then, Olivia and Ray had lots of fun at the ra...",Olivia,Ray,ABB,3,5,6,14,18
4,"Then, Christopher and Noah were working at the...",Christopher,Noah,ABB,3,5,6,12,18


In [6]:
circuit.get_activations(128)

100%|██████████| 157/157 [00:15<00:00, 10.08it/s]


In [7]:
circuit.compute_supervised_dictionary()

100%|██████████| 216/216 [00:19<00:00, 11.07it/s]


In [9]:
circuit.IO_features[' Adam']['k']['IO'].shape

torch.Size([12, 12, 64])

In [5]:
import torch
from tqdm import tqdm

bs = 64

activations = {i: [] for i in ['q', 'k', 'v', 'z']}
for b in tqdm(range(0, len(task_df), bs)):
    tokens = model.to_tokens(task_df.iloc[b:b+bs, 0])

    positions = task_df.iloc[b:b+bs][['IO_pos', 'S1_pos', 'S1+1_pos', 'S2_pos', 'END']].values
    
    with torch.no_grad():
        _, cache = model.run_with_cache(tokens)

    for key in activations.keys():
        key_acts = cache.stack_activation(key) # [l b pos h dm]
        activations[key].append(torch.cat([key_acts[:, None, i, pos] for i, pos in enumerate(positions)], dim=1))

    del cache

activations = {key: torch.cat(values, 1).cpu() for key, values in activations.items()} # [l N 5 h dm]
mean_activations = {key: torch.mean(values, 1).cpu() for key, values in activations.items()} # [l 5 h dm]

100%|██████████| 313/313 [00:18<00:00, 16.56it/s]


In [31]:
def supervised_dictionary(df, activations):
    
    io_vec = {}
    s_vec = {}

    attributes = ['IO', 'S1', 'S1+1', 'S2', 'END']

    unique_io = df['IO'].unique()
    unique_s1 = df['S1'].unique()

    centered_activations = {c: activations[c] - activations[c].mean(1)[:, None] for c in ['q', 'k', 'v', 'z']}

    for name in tqdm(set(unique_io) | set(unique_s1)):
        io_vec[name] = {}
        s_vec[name] = {}

        for c in ['q', 'k', 'v', 'z']:
            mask = torch.tensor(df['IO'] == name, dtype=torch.bool, device=activations[c].device)
            if mask.any():
                filtered_activations = centered_activations[c][:, mask] # [l M 5 h dm]
                io_vec[name][c] = {a: filtered_activations[:, :, i].mean(1) for i, a in enumerate(attributes)}

            mask = torch.tensor(df['S1'] == name, dtype=torch.bool, device=activations[c].device)
            if mask.any():
                filtered_activations = centered_activations[c][:, mask]
                s_vec[name][c] = {a: filtered_activations[:, :, i].mean(1) for i, a in enumerate(attributes)}

    # Pos
    pos_vec = {'ABB': {}, 'BAB': {}}
    
    for c in ['q', 'k', 'v', 'z']:
        for pos in [0, 1]:
            mask = torch.tensor(df['POS'] == pos, dtype=torch.bool, device=activations[c].device)
            if mask.any():
                filtered_activations = centered_activations[c][:, mask]
                pos_vec['ABB' if pos == 0 else 'BAB'][c] = {a: filtered_activations[:, :, i].mean(1) for i, a in enumerate(attributes)}
    
    return (io_vec, s_vec, pos_vec)

In [32]:
f = supervised_dictionary(task_df, activations)

100%|██████████| 216/216 [00:19<00:00, 11.27it/s]


In [33]:
f[0][' Richard']['q']['IO'].shape

torch.Size([12, 12, 64])

In [3]:
circuit.prompts[0].text, circuit.prompts[0].generate_corrupted('POS').get_variable('S')

TypeError: IOIPrompt.__init__() missing 1 required positional argument: 'model'

In [14]:
for p in prompts:
    print(p)
    p['variables']['S1+1'] = p['variables']['S1_pos'] + 1

{'prompt': 'Then, Bella and John had lots of fun at the clinic. John gave a pillow to', 'variables': {'IO': ' Bella', 'POS': 'ABB', 'END': 18, 'IO_pos': 3, 'S1_pos': 5, 'S2_pos': 14, 'S': ' John'}}


KeyError: 'S1'

In [17]:
!nvidia-smi

Fri Jun 28 16:24:21 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.154.05             Driver Version: 535.154.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA RTX A6000               On  | 00000000:00:07.0 Off |                  Off |
| 30%   50C    P2              87W / 300W |  26212MiB / 49140MiB |     15%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--